In [ ]:
# for reading and validating data
import emeval.input.spec_details as eisd
import emeval.input.phone_view as eipv
import emeval.input.eval_view as eiev

In [ ]:
# Visualization helpers
import emeval.viz.phone_view as ezpv
import emeval.viz.eval_view as ezev

In [ ]:
# for pipelined data
import emeval.analysed.phone_view as eapv

In [ ]:
# for speed
import emeval.analysed.location_smoothing as eals

In [ ]:
import pandas as pd

In [ ]:
import geopandas as gpd
import shapely as shp

In [ ]:
import numpy as np

In [ ]:
import emeval.metrics.dist_calculations as emd

In [ ]:
# For plots
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# Analytics results
import emeval.metrics.segmentation as ems

In [ ]:
# For easier debugging while working on modules
import importlib

In [ ]:
def import_sd_and_pv_from_server(trips  = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"], 
                                 AUTHOR_EMAIL  = "shankari@eecs.berkeley.edu", 
                                 DATASTORE_LOC = "http://localhost:8080", 
                                 pkl_file_name = None):
    sd_l = []
    pv_l = []
    for trip in trips:
        sd = eisd.ServerSpecDetails(DATASTORE_LOC, AUTHOR_EMAIL, trip)
        pv = eipv.PhoneView(sd)
        sd_l.append(sd)
        pv_l.append(pv)
    if pkl_file_name:
        import pickle
        with open(pkl_file_name, 'wb') as outp:
            for pv in pv_l:
                pickle.dump(pv, outp, pickle.HIGHEST_PROTOCOL)
    return sd_l, pv_l

In [ ]:
def import_pv_from_pkl(pkl_file_name, 
                       trips = ["unimodal_trip_car_bike_mtv_la", "car_scooter_brex_san_jose", "train_bus_ebike_mtv_ucb"]):
    import pickle
    pv_l = []
    with open('pv.pkl', 'rb') as inp:
        for trip in trips:
            pv_l.append(pickle.load(inp))
    return pv_l

In [ ]:
(pv_la, pv_sj, pv_ucb) = import_pv_from_pkl('pv.pkl')

In [ ]:
av_la = eapv.create_analysed_view(pv_la, "http://localhost:8080", "analysis/recreated_location", "analysis/cleaned_trip", "analysis/cleaned_section")

In [ ]:
importlib.reload(eals)

In [ ]:
pv = av_la
for phone_os, phone_map in pv.map().items():
    for phone_label, phone_detail_map in phone_map.items():
        for r in phone_detail_map["evaluation_ranges"]:
            for tr in r["evaluation_trip_ranges"]:
                for ss in tr['sensed_section_ranges']:
                    pass
                for sr in tr['evaluation_section_ranges']:
                    section_gt_leg = pv.spec_details.get_ground_truth_for_leg(tr["trip_id_base"], sr["trip_id_base"], tr['start_ts'], tr['end_ts'])
                    section_gt_shapes = gpd.GeoSeries(eisd.SpecDetails.get_shapes_for_leg(section_gt_leg))
                    section_gt_shapes_df = pd.DataFrame(section_gt_shapes)
                    eals.add_dist_heading_speed(section_gt_shapes_df)
                    1/0